
<div  style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/derar-alhussein/Databricks-Certified-Data-Engineer-Associate/main/Includes/images/bookstore_schema.png" alt="Databricks Learning" style="width: 600">
</div>

# Configurar Serverless

Antes de ejecutar el motor Serverless debes cargar las librerías

1. Clic en => Environment en el panel de la derecha
2. En la sección Dependencies carga el archivo requirements que se encuentra en la raíz del proyecto
3. Presiona apply

# Generación de datos en tiempo real
Al ejecutar 

```python
parameters = {"volume_path": "/volume/folder"}
dbutils.notebook.run("../Includes/GenDataStream", 3600, parameters)
```

Usted debe proporcionar la ruta del volumen donde se van a almacenar los datos generados bajo demanda (cada vez que ejecute el comando se van a generar nuevos datos)

Para poder crear un volumen siga estas instrucciones:

```sql
-- Crear un catalogo
CREATE CATALOG IF NOT EXISTS [catalogo]
-- Crear un esquema 
CREATE DATABASE IF NOT EXISTS [catalogo].[esquema]
-- Crear un volumen
CREATE VOLUME IF NOT EXISTS [catalogo].[esquema].[volumen]

```

Mediante el navegador de catálogos de Databricks ubíquese en el volumen y copie la ruta, y guardelo como valor del widget llamado volume_path

Ejemplo:

```sh
/Volumes/catalogo/esquema/volumen
```


In [0]:
dbutils.widgets.text("volume_path","")

In [0]:
%python
volume_path=dbutils.widgets.get("volume_path")
if volume_path=="":
  raise Exception("Debe definir una ruta de volumen")

In [0]:
dbutils.notebook.run("../Includes/GenDataStream", 3600, {"volume_path": volume_path})


## Exploring The Source Directory

In [0]:
files = dbutils.fs.ls(f"{volume_path}/orders")
display(files)


## Auto Loader

Si usas Serverless no vas a poder realizar ejecuciones continuas. Entonces deberás usar All Purpose o Job Cluster para cambiar el trigger a ejecuciones permanentes como por ejemplo:

```python
.trigger(processingTime="2 seconds")
```

In [0]:
(spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option("cloudFiles.schemaLocation",f"{volume_path}/schema_cache")
        .load(f"{volume_path}/orders")
      .writeStream
        .option("checkpointLocation", f"{volume_path}/checkpoint")
        .trigger(availableNow=True)
        .table("orders_updates")
)

In [0]:
%sql
SELECT * FROM orders_updates

In [0]:
%sql
SELECT count(*) FROM orders_updates


## Landing New Files

In [0]:
# Load new data
dbutils.notebook.run("../Includes/GenDataStream", 3600, {"volume_path": volume_path})

In [0]:
files = dbutils.fs.ls(f"{volume_path}/orders")
display(files)

In [0]:
(spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "parquet")
        .option("cloudFiles.schemaLocation",f"{volume_path}/schema_cache")
        .load(f"{volume_path}/orders")
      .writeStream
        .option("checkpointLocation", f"{volume_path}/checkpoint")
        .trigger(availableNow=True)
        .table("orders_updates")
)

In [0]:
%sql
SELECT count(*) FROM orders_updates


## Exploring Table History

In [0]:
%sql
DESCRIBE HISTORY orders_updates


## Cleaning Up

In [0]:
%sql
DROP TABLE orders_updates

In [0]:
dbutils.fs.rm(f"{volume_path}/schema_cache", True)
dbutils.fs.rm(f"{volume_path}/checkpoint", True)
dbutils.fs.rm(f"{volume_path}/orders", True)